# Build Medical Imaging Extension

### Prerequisites
* Download ODBC Driver 18 from web <https://learn.microsoft.com/en-us/sql/connect/odbc/linux-mac/install-microsoft-odbc-driver-sql-server-macos?view=sql-server-ver16>
* OMOP CDM instance with the connection string and authentication information.
* Install SQL processing package, i.e., pyodbc.

## Tables to create
1. Image_occurrence
2. Image_feature

In [2]:
import psycopg2
conn = psycopg2.connect(
    database="postgres",
    user="",
    password="",
    host="",
    port="5432",
    connect_timeout = 6000
)
cursor = conn.cursor()

In [3]:
# Create Image_occurrence table

# postgre sql does not have varchar(max) so we will use varchar(10000)

ddl_statement = """
CREATE TABLE adni.image_occurrence(
    image_occurrence_id integer NOT NULL,
    person_id integer NOT NULL,
    procedure_occurrence_id integer NOT NULL,
    visit_occurrence_id integer,
    anatomic_site_concept_id integer,
    wadors_uri varchar(10000),
    local_path varchar(10000),
    image_occurrence_date date NOT NULL,
    image_study_uid varchar(10000) NOT NULL,
    image_series_uid varchar(10000) NOT NULL,
    modality_concept_id integer NOT NULL
);
"""

cursor = conn.cursor()
cursor.execute(ddl_statement)
conn.commit()

In [4]:
# Create Image_feature table

# postgre sql does not have varchar(max) so we will use varchar(10000)
# also don't have datetime datatype so we will use timestamp

ddl_statement = """
CREATE TABLE adni.image_feature(
    image_feature_id integer NOT NULL,
    person_id integer NOT NULL,
    image_occurrence_id integer NOT NULL,
    image_feature_event_field_concept_id integer,
    image_feature_event_id integer,
    image_feature_concept_id integer NOT NULL,
    image_feature_type_concept_id integer NOT NULL,
    image_finding_concept_id integer,
    image_finding_id integer,
    anatomic_site_concept_id integer,
    alg_system varchar(10000),
    alg_datetime timestamp
);
"""

cursor.execute(ddl_statement)
conn.commit()

# Clean up
cursor.close()
conn.close()